In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# Carga de Datos

In [4]:
dataframe = pd.read_excel("../data/raw/15_3_01_Energia_electrica_consumida_por_tipo_usuario-5.xlsx", sheet_name="Total TDF", header=3)

# Exploración inicial de datos))

In [6]:
# Visualización  de cantidad de instancias y columnas.
num_instancias, num_columnas = dataframe.shape
print(f'Cantidad de instancias: {num_instancias}')
print(f'Cantidad de columnas: {num_columnas}')

Cantidad de instancias: 182
Cantidad de columnas: 8


In [8]:
# Mostrar las primeras filas para verificar la correcta carga del archivo
dataframe.head()

,Unnamed: 0,Período,Total,Grandes \nDemandas,Medianas \nDemandas,Uso\n Residencial,Uso \nGeneral,Alumbrado \nPúblico
0,NaN,NaN,kWh,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Enero,28389813,14295431,1895533,8505506,2752545,940798
3,NaN,Febrero,29604832,14258851,2454954,7957003,3910403,1023621
4,NaN,Marzo,31100802,16086916,2404916,8382174,2837123,1389673


In [10]:
# Verificar tipo de datos
print(dataframe.dtypes)

Unnamed: 0              object
Período                 object
Total                   object
Grandes \nDemandas      object
Medianas \nDemandas     object
Uso\n Residencial       object
Uso \nGeneral           object
Alumbrado \nPúblico     object
dtype: object


In [14]:
dataframe.isnull().sum()

Unnamed: 0              164
Período                   7
Total                     6
Grandes \nDemandas        7
Medianas \nDemandas       7
Uso\n Residencial         7
Uso \nGeneral             7
Alumbrado \nPúblico       7
dtype: int64

In [16]:
# Mostrar la información general del dataset
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           18 non-null     object
 1   Período              175 non-null    object
 2   Total                176 non-null    object
 3   Grandes 
Demandas    175 non-null    object
 4   Medianas 
Demandas   175 non-null    object
 5   Uso
 Residencial     175 non-null    object
 6   Uso 
General         175 non-null    object
 7   Alumbrado 
Público   175 non-null    object
dtypes: object(8)
memory usage: 11.5+ KB


# Correción y limpieza del Dataset

In [18]:
# Eliminar las filas 1 y 2 (índices 0 y 1)
dataframe = dataframe.drop([0, 1]).reset_index(drop=False)
dataframe.head()

,index,Unnamed: 0,Período,Total,Grandes \nDemandas,Medianas \nDemandas,Uso\n Residencial,Uso \nGeneral,Alumbrado \nPúblico
0,2,2010,Enero,28389813,14295431,1895533,8505506,2752545,940798
1,3,NaN,Febrero,29604832,14258851,2454954,7957003,3910403,1023621
2,4,NaN,Marzo,31100802,16086916,2404916,8382174,2837123,1389673
3,5,NaN,Abril,34441839,15568924,2991200,9663763,4604950,1613002
4,6,NaN,Mayo,32849499,15230025,2664536,9744634,3245695,1964609


In [30]:
# Renombrar características
dataframe = dataframe.rename(columns={
    "Unnamed: 0": "Año",
    "Grandes \nDemandas": "Grandes Demandas",
    "Medianas \nDemandas": "Medianas Demandas",
    "Uso\n Residencial": "Uso Residencial",
    "Uso \nGeneral": "Uso General",
    "Alumbrado \nPúblico ": "Alumbrado Público"
})

print(dataframe.columns)

Index(['index', 'Año', 'Período', 'Total', 'Grandes Demandas',
       'Medianas Demandas', 'Uso Residencial', 'Uso General',
       'Alumbrado Público'],
      dtype='object')


In [34]:
# Completar instancias de año vacias con el año correspondiente
dataframe["Año"] = dataframe["Año"].ffill()

In [36]:
# Renombrar los nombres de meses con números enteros. 
meses_dict = {
    "Enero": 1,
    "Febrero": 2,
    "Marzo": 3,
    "Abril": 4,
    "Mayo": 5,
    "Junio": 6,
    "Julio": 7,
    "Agosto": 8,
    "Septiembre": 9,
    "Octubre": 10,
    "Noviembre": 11,
    "Diciembre": 12
}
dataframe["Período"] = dataframe["Período"].replace(meses_dict)

In [38]:
# Visualizar el tipo de dato de cada columna
print(dataframe.dtypes)

index                 int64
Año                  object
Período              object
Total                object
Grandes Demandas     object
Medianas Demandas    object
Uso Residencial      object
Uso General          object
Alumbrado Público    object
dtype: object


In [42]:
# Calcula el numero de valores nullos de cada columna
null_counts = dataframe.isnull().sum()
print(null_counts)

index                0
Año                  0
Período              5
Total                5
Grandes Demandas     5
Medianas Demandas    5
Uso Residencial      5
Uso General          5
Alumbrado Público    5
dtype: int64


In [44]:
# Vemos las instancias nulas almacenadas en "filas_nulas"
filas_nulas = dataframe[dataframe.isnull().any(axis=1)]
print(filas_nulas)

     index                                                Año Período Total  \
175    177                                               2024     NaN   NaN   
176    178  Nota: Los años 2010 a 2016 no incluyen el cons...     NaN   NaN   
177    179  ...Dato no disponible a la fecha de presentaci...     NaN   NaN   
178    180  ...Dato no disponible a la fecha de presentaci...     NaN   NaN   
179    181  Fuente: Dirección Provincial de Energía Ushuai...     NaN   NaN   

    Grandes Demandas Medianas Demandas Uso Residencial Uso General  \
175              NaN               NaN             NaN         NaN   
176              NaN               NaN             NaN         NaN   
177              NaN               NaN             NaN         NaN   
178              NaN               NaN             NaN         NaN   
179              NaN               NaN             NaN         NaN   

    Alumbrado Público  
175               NaN  
176               NaN  
177               NaN  
178     

In [46]:
# Eliminar las filas nulas del DataFrame original
dataframe = dataframe.drop(filas_nulas.index)

In [48]:
null_counts = dataframe.isnull().sum()
print(null_counts)

index                0
Año                  0
Período              0
Total                0
Grandes Demandas     0
Medianas Demandas    0
Uso Residencial      0
Uso General          0
Alumbrado Público    0
dtype: int64


In [58]:
# Verificar nuevamente cantidad de instancias y columnas
print("Cantidad de instancias y columnas:", dataframe.shape)

Cantidad de instancias y columnas: (175, 9)


In [ ]:
####
### continuar desde aqui en adelante
###

In [ ]:
# Supongamos que estás trabajando con la columna 'Total'
problematic_rows = dataframe[~dataframe['Total'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
print(problematic_rows)


In [ ]:
# Eliminar las filas problemáticas
dataframe = dataframe[dataframe['Total'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que tu DataFrame se llama 'dataframe'
sns.boxplot(x=dataframe['Total'])
plt.title('Boxplot de Total')
plt.show()

In [ ]:
pip install openpyxl

In [ ]:
# Guardar el DataFrame como archivo CSV
dataframe.to_csv('Consumo_Energia_Electrica_v1.csv', index=False)

# ME DA ERROR AL GUARDAR ALGUNOS DATOS, MESES NO GUARDA CON NUM , ALGUNOS EN LETRAS. SEGUIR LIMPIANDO EL DATASET